# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Adding this to hide the pink warnings after they have been viewed once
import warnings
warnings.filterwarnings('ignore')

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../WeatherPy/output_data/cities.csv"

weather_df = pd.read_csv(file)
weather_df.head()

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
0,Busselton,AU,57.24,1613157827,89,0,11.54,-33.6500,115.3333
1,Lompoc,US,54.23,1613157828,62,20,17.27,34.6391,-120.4579
2,Nikolskoye,RU,0.68,1613157829,78,75,6.71,59.7035,30.7861
3,Adavere,EE,6.93,1613157830,85,90,3.44,58.7086,25.8994
4,Alofi,NU,78.80,1613157831,94,99,6.91,-19.0595,-169.9187


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Use the lat and lng columns as the locations variable
locations = weather_df[["Lat", "Lng"]]

# Use the humidity % column of the df as the weight variable
humidity = weather_df["Humidity %"]

In [4]:
# Generate the map
# From class discussion, it is important to set a center, and a zoom level to help make the map display properly
# Got the center and zoom_level values through trial and error
fig = gmaps.figure(center=(25.0, 15.0), zoom_level = 1.8)

# Generate a new layer on the map - Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Per homework instructions, I will narrow down to cities with the following:
# Temp 80 > and < 70, Wind Speed < 10, Cloudiness = 0
# Actually, going to be a little more lax on "perfect" so that I can get more results for later in the assignment (And I like it a little warmer)


# Start by dropping NA values
weather_df.dropna(inplace = True)

# Start by finding cities with the perfect temperature (Temp 80 > and < 70)
# Switching to 85 for the high to improve results
perfect_temperature = weather_df.loc[(weather_df["Temperature (F)"] < 85) & (weather_df["Temperature (F)"] > 70)]
perfect_temperature

# Next narrow it down further by looking for lower wind speeds (< 10)
less_wind = perfect_temperature.loc[(perfect_temperature["Wind Speed"] < 10)]
less_wind

# Finally remove all clouds in the sky by setting cloudiness equal to zero
# Switching to a little cloudy to improve results (and because I don't mind some shade)
# This creates our perfect weather dataframe
perfect_weather = less_wind.loc[(less_wind["Cloud"] <= 15)]
perfect_weather

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
30,Avarua,CK,82.4,1613157895,74,9,2.30,-21.2078,-159.7750
42,Hilo,US,73.6,1613157784,50,1,2.17,19.7297,-155.0900
87,Bredasdorp,ZA,71.6,1613158044,73,1,6.91,-34.5322,20.0403
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667
225,Luanda,AO,80.6,1613158346,78,0,9.22,-8.8368,13.2343
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create hotel_df based on the perfect weather df above
hotel_df = perfect_weather
hotel_df

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng
30,Avarua,CK,82.4,1613157895,74,9,2.30,-21.2078,-159.7750
42,Hilo,US,73.6,1613157784,50,1,2.17,19.7297,-155.0900
87,Bredasdorp,ZA,71.6,1613158044,73,1,6.91,-34.5322,20.0403
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667
225,Luanda,AO,80.6,1613158346,78,0,9.22,-8.8368,13.2343
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924


In [7]:
# Adding column "Hotel Name"
# Because I don't have the values for that column yet, I will set it equal to quotes to leave it blank (ignore pink message box)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng,Hotel Name
30,Avarua,CK,82.4,1613157895,74,9,2.30,-21.2078,-159.7750,
42,Hilo,US,73.6,1613157784,50,1,2.17,19.7297,-155.0900,
87,Bredasdorp,ZA,71.6,1613158044,73,1,6.91,-34.5322,20.0403,
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662,
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667,
225,Luanda,AO,80.6,1613158346,78,0,9.22,-8.8368,13.2343,
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924,


In [8]:
# Prepare the api requests by setting up the url and parameters
# Using the "nearby search" type to perform this request
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Parameters for search based on hw and (https://developers.google.com/places/web-service/supported_types)
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [9]:
# Run API request to find the hotels within 5,000 meters of the cities in our dataframe

# Even though this is a small dataframe, I am still going to use a counter here to keep track
counter = 0

# Use for loop to run through each row in the dataframe
for index, perfect_city in hotel_df.iterrows():
    
    # Increase counter as we move through to the each row in the dataframe
    counter += 1
    
    # Pull the city name, lat, and lng values from the row in hotel_df
    lat = perfect_city["Lat"]
    lng = perfect_city["Lng"]
    city_name = perfect_city["City"]
    
    # add keyword to params dict
    params["location"] = (f"{lat}, {lng}")

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(url, params)
    # Convert to json
    response_json = response.json()
    
    # Extract the results and store them in a well-named variable
    results = response_json["results"]
    
    # Print statement to show where we are in the counting process of the dataframe search
    print(f"{counter} of {len(hotel_df)}: Looking for hotels near {city_name}")
    #print(json.dumps(results, indent = 4, sort_keys = True))   
    
    # If possible save the hotel name to dataframe
    # Need to use try logic hear so that even if it doesn't find a value for one city, it can move on and find the others
    try:
        hotel_name = results[0]["name"]
        print(f"Hotel found! {hotel_name}")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]

    # Display a message if unable to find a hotel with the given parameters
    except (KeyError, IndexError):
        print(f"Sorry, we could not find a hotel near {city_name}")
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
    # Adding in a one second sleep interval between queries to avoid exceeding the API query limits
    time.sleep(1)
    
# Print end of search once searching is completed
print("-----------------------------")
print(f"Finished collecting hotel information from {len(hotel_df)} cities with perfect weather conditions")

hotel_df

1 of 7: Looking for hotels near Avarua
Hotel found! The Islander Hotel
2 of 7: Looking for hotels near Hilo
Hotel found! Hilo Hawaiian Hotel
3 of 7: Looking for hotels near Bredasdorp
Hotel found! Bredasdorp Country Manor
4 of 7: Looking for hotels near Coyhaique
Hotel found! Patagonia House
5 of 7: Looking for hotels near Jizan
Sorry, we could not find a hotel near Jizan
6 of 7: Looking for hotels near Luanda
Hotel found! EPIC SANA Luanda
7 of 7: Looking for hotels near Salalah
Hotel found! HYATT SALALAH HOTEL
-----------------------------
Finished collecting hotel information from 7 cities with perfect weather conditions


,City,Country,Temperature (F),Date,Humidity %,Cloud,Wind Speed,Lat,Lng,Hotel Name
30,Avarua,CK,82.4,1613157895,74,9,2.30,-21.2078,-159.7750,The Islander Hotel
42,Hilo,US,73.6,1613157784,50,1,2.17,19.7297,-155.0900,Hilo Hawaiian Hotel
87,Bredasdorp,ZA,71.6,1613158044,73,1,6.91,-34.5322,20.0403,Bredasdorp Country Manor
89,Coyhaique,CL,75.2,1613158020,29,0,6.91,-45.5752,-72.0662,Patagonia House
100,Jizan,SA,78.8,1613158067,73,0,3.44,17.3333,42.6667,NA
225,Luanda,AO,80.6,1613158346,78,0,9.22,-8.8368,13.2343,EPIC SANA Luanda
242,Salalah,OM,73.4,1613158396,49,0,3.44,17.0151,54.0924,HYATT SALALAH HOTEL


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))